In [ ]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()  # charge les variables depuis .env

client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
)

# 📄 Azure Content Understanding — Prebuilt Analyzer Test

**Service** : Azure AI Content Understanding (REST API `2025-11-01`)  
**Auth** : Azure AD (DefaultAzureCredential) — key auth disabled on this resource  
**Plan** :  
1. Acquire token once (cached)  
2. Test available prebuilt analyzers on 1 doc  
3. Run the best ones on 20 docs from `batch1_1`  
4. Save JSON results to `docu_results_batch1_1/`

In [ ]:
%pip install requests python-dotenv
%pip install cryptography --only-binary=:all:
%pip install azure-identity

In [8]:
import os, json, time, glob, base64, requests
from azure.identity import DefaultAzureCredential

# ── Config ──
ENDPOINT    = "https://aya-demo-ai.cognitiveservices.azure.com"
API_VERSION = "2025-11-01"
DOCS_FOLDER  = r"c:\Users\ayac\Downloads\archive (1)\batch_1\batch_1\batch1_1"
OUTPUT_FOLDER = r"c:\Users\ayac\Downloads\archive (1)\batch_1\batch_1\docu_results_batch1_1"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ── Azure AD Token (cached) ──
credential = DefaultAzureCredential()
_token = credential.get_token("https://cognitiveservices.azure.com/.default")

def auth():
    global _token
    if time.time() > _token.expires_on - 120:
        _token = credential.get_token("https://cognitiveservices.azure.com/.default")
    return {"Authorization": f"Bearer {_token.token}"}

# ── First 20 images ──
all_images = sorted(glob.glob(os.path.join(DOCS_FOLDER, "*.jpg")))[:20]
print(f"🔑 Token OK")
print(f"🌐 Endpoint: {ENDPOINT}")
print(f"📂 {len(all_images)} documents ready")

🔑 Token OK
🌐 Endpoint: https://aya-demo-ai.cognitiveservices.azure.com
📂 20 documents ready


## 🧪 Step 1 — Test which prebuilt analyzers work on this resource

Quick test on 1 document to identify working analyzers before running the full batch.

In [9]:
def submit(image_path, analyzer_id):
    """POST image to Content Understanding → returns operation URL."""
    with open(image_path, "rb") as f:
        data = base64.b64encode(f.read()).decode("utf-8")
    url = f"{ENDPOINT}/contentunderstanding/analyzers/{analyzer_id}:analyze?api-version={API_VERSION}"
    r = requests.post(url, headers={**auth(), "Content-Type": "application/json"},
                      json={"inputs": [{"data": data, "mimeType": "image/jpeg"}]})
    if r.status_code != 202:
        raise RuntimeError(f"{r.status_code}: {r.text[:500]}")
    return r.headers["Operation-Location"]

def poll(op_url, timeout=600):
    """Poll operation URL until Succeeded (default 10 min timeout)."""
    for i in range(timeout // 5):
        time.sleep(5)
        r = requests.get(op_url, headers=auth())
        r.raise_for_status()
        res = r.json()
        st = res.get("status", "")
        if st == "Succeeded": return res
        if st in ("Failed","Canceled"):
            raise RuntimeError(json.dumps(res.get("error", res), indent=2))
        if i % 6 == 5:
            print(f"    ...still waiting ({(i+1)*5}s, status={st})", flush=True)
    raise TimeoutError("Timeout")

# ── Test just prebuilt-invoice first (may take time on first call) ──
test_img = all_images[0]
print(f"🧪 Testing prebuilt-invoice on {os.path.basename(test_img)}...")
print(f"   (first call may take 3-6 min while models deploy)\n")

t0 = time.time()
try:
    op = submit(test_img, "prebuilt-invoice")
    print(f"   ✓ Submitted (202). Polling (10 min timeout)...")
    res = poll(op, timeout=600)
    dt = time.time() - t0
    c = res.get("result",{}).get("contents",[])
    block = c[0] if c else {}
    fields = block.get("fields",{})
    md = block.get("markdown","")
    print(f"\n   ✅ prebuilt-invoice SUCCESS! ({dt:.0f}s)")
    print(f"      Fields: {len(fields)} | Markdown: {len(md)} chars")
    print(f"      Field names: {list(fields.keys())[:10]}")
    ANALYZERS = ["prebuilt-invoice"]
except Exception as e:
    dt = time.time() - t0
    print(f"\n   ❌ prebuilt-invoice failed ({dt:.0f}s): {e}")
    ANALYZERS = ["prebuilt-layout", "prebuilt-read"]
    print(f"   Falling back to: {ANALYZERS}")

print(f"\n📌 Analyzers for batch: {ANALYZERS}")

🧪 Testing prebuilt-invoice on batch1-0001.jpg...
   (first call may take 3-6 min while models deploy)

   ✓ Submitted (202). Polling (10 min timeout)...

   ✅ prebuilt-invoice SUCCESS! (8s)
      Fields: 31 | Markdown: 13 chars
      Field names: ['AmountDue', 'BalanceForward', 'BillingAddress', 'BillingAddressRecipient', 'CountryRegion', 'CustomerAddress', 'CustomerAddressRecipient', 'CustomerId', 'CustomerName', 'CustomerTaxId']

📌 Analyzers for batch: ['prebuilt-invoice']


In [10]:
# Check if the previous prebuilt-invoice job finished by now
# (it was submitted ~5 min ago, may have completed since)
print("🔍 Checking if previous prebuilt-invoice job completed...\n")

# Re-submit a fresh one and poll with very long timeout
t0 = time.time()
op = submit(all_images[0], "prebuilt-invoice")
print(f"   Submitted. Polling with 10 min timeout...")

try:
    res = poll(op, timeout=600)
    dt = time.time() - t0
    c = res.get("result",{}).get("contents",[])
    block = c[0] if c else {}
    fields = block.get("fields",{})
    md = block.get("markdown","")
    print(f"\n✅ prebuilt-invoice works! ({dt:.0f}s)")
    print(f"   Fields: {len(fields)} | Markdown: {len(md)} chars")
    print(f"   Field names: {list(fields.keys())}")
    
    # Save this test result
    os.makedirs(os.path.join(OUTPUT_FOLDER, "prebuilt-invoice"), exist_ok=True)
    with open(os.path.join(OUTPUT_FOLDER, "prebuilt-invoice", "batch1-0001.json"), "w", encoding="utf-8") as f:
        json.dump(res, f, indent=2, ensure_ascii=False)
    
    ANALYZERS = ["prebuilt-invoice"]
    print(f"\n📌 Will use: {ANALYZERS} for the batch!")
except Exception as e:
    dt = time.time() - t0
    print(f"\n❌ Still failing ({dt:.0f}s): {e}")
    print("   The GPT models may still be deploying on your resource.")
    print("   You can check deployment status in Azure Portal → aya-demo-ai → Model deployments")
    ANALYZERS = ["prebuilt-layout", "prebuilt-read"]
    print(f"\n📌 Falling back to: {ANALYZERS}")

🔍 Checking if previous prebuilt-invoice job completed...

   Submitted. Polling with 10 min timeout...

✅ prebuilt-invoice works! (7s)
   Fields: 31 | Markdown: 13 chars
   Field names: ['AmountDue', 'BalanceForward', 'BillingAddress', 'BillingAddressRecipient', 'CountryRegion', 'CustomerAddress', 'CustomerAddressRecipient', 'CustomerId', 'CustomerName', 'CustomerTaxId', 'DueDate', 'InvoiceDate', 'InvoiceId', 'LineItems', 'PaymentTerm', 'PONumber', 'RemittanceAddress', 'RemittanceAddressRecipient', 'ServiceAddress', 'ServiceAddressRecipient', 'ShippingAddress', 'ShippingAddressRecipient', 'SubtotalAmount', 'TaxDetails', 'TotalAmount', 'TotalDiscountAmount', 'TotalTaxAmount', 'VendorAddress', 'VendorAddressRecipient', 'VendorName', 'VendorTaxId']

📌 Will use: ['prebuilt-invoice'] for the batch!


## 🚀 Step 2 — Batch analysis: submit ALL, then poll ALL

Submits all requests first (fast), then collects results — much faster than sequential.

In [ ]:
all_metrics = {a: [] for a in ANALYZERS}
jobs = []  # (img_path, analyzer, op_url, t0)

# ── Helper: extract field values (name → value) recursively ──
def extract_field_values(fields_dict):
    """Flatten fields into {name: value} dict.
    Only keeps fields that have actual values (valueString, valueNumber, etc.),
    skips fields that only have schema definitions (type, no value)."""
    result = {}
    for name, obj in fields_dict.items():
        if not isinstance(obj, dict):
            continue
        # Skip fields that only have schema (type) but no actual value
        if set(obj.keys()) <= {"type", "valueObject"} and "valueObject" in obj:
            # Check if sub-fields have actual values
            sub = extract_field_values(obj["valueObject"])
            if sub:  # only add if sub-fields have real values
                result[name] = sub
        elif set(obj.keys()) <= {"type"}:
            continue  # schema-only field, skip
        else:
            # Field has actual value
            val = obj.get("valueString") or obj.get("valueNumber") or obj.get("valueDate") or obj.get("content") or obj.get("value")
            if val is not None:
                result[name] = val
            elif "valueObject" in obj:
                sub = extract_field_values(obj["valueObject"])
                if sub:
                    result[name] = sub
            elif "valueArray" in obj:
                arr = []
                for item in obj["valueArray"]:
                    if isinstance(item, dict) and "valueObject" in item:
                        sub = extract_field_values(item["valueObject"])
                        if sub:
                            arr.append(sub)
                    elif isinstance(item, dict):
                        v = item.get("valueString") or item.get("content") or item.get("value")
                        if v:
                            arr.append(v)
                if arr:
                    result[name] = arr
    return result

# ── Helper: LLM description via GPT-4.1 Vision (multimodal) ──
GPT_DEPLOYMENT = "gpt-4.1"

def llm_describe(image_path, fname):
    """Send the actual image to GPT-4.1 Vision for a global document description."""
    with open(image_path, "rb") as f:
        img_b64 = base64.b64encode(f.read()).decode("utf-8")

    url = f"{ENDPOINT}/openai/deployments/{GPT_DEPLOYMENT}/chat/completions?api-version=2024-12-01-preview"
    body = {
        "messages": [
            {"role": "system", "content": "Tu es un assistant d'analyse documentaire expert. Tu analyses des documents scannés (factures, devis, bons de commande, etc.) et tu fournis une description globale concise."},
            {"role": "user", "content": [
                {"type": "text", "text": f"Analyse cette image du document \"{fname}\". Donne une description globale en 2-3 phrases en français: quel type de document est-ce, qui est l'émetteur, le destinataire, le montant total, la date, et tout élément clé."},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_b64}", "detail": "high"}}
            ]}
        ],
        "max_tokens": 300,
        "temperature": 0.3,
    }
    try:
        r = requests.post(url, headers={**auth(), "Content-Type": "application/json"}, json=body)
        r.raise_for_status()
        return r.json()["choices"][0]["message"]["content"].strip()
    except Exception as e:
        return f"[LLM error: {e}]"

# ── Phase 1: Submit all ──
print("📤 Submitting all requests...")
for i, img in enumerate(all_images, 1):
    fname = os.path.basename(img)
    for aid in ANALYZERS:
        try:
            op = submit(img, aid)
            jobs.append((img, aid, op, time.time()))
            print(f"  ✓ [{i}/20] {fname} → {aid}")
        except Exception as e:
            print(f"  ✗ [{i}/20] {fname} → {aid}: {e}")
            all_metrics[aid].append({"document": fname, "analyzer": aid,
                                     "error": str(e), "time_seconds": 0})

print(f"\n📤 {len(jobs)} jobs submitted. Waiting 8s before polling...")
time.sleep(8)

# ── Phase 2: Collect results ──
print("\n📥 Collecting results...")
done = 0
for img, aid, op_url, t0 in jobs:
    fname = os.path.basename(img)
    dname = os.path.splitext(fname)[0]
    done += 1
    try:
        res = poll(op_url)
        dt = time.time() - t0
        c = res.get("result",{}).get("contents",[])
        block = c[0] if c else {}
        fields = block.get("fields", {})
        md = block.get("markdown", "")

        # Extract field values (only real values, skip schema-only)
        field_values = extract_field_values(fields)

        # Count field confidences recursively
        confs = []
        def _gc(o):
            if isinstance(o, dict):
                if "confidence" in o: confs.append(o["confidence"])
                for v in o.values(): _gc(v)
            elif isinstance(o, list):
                for x in o: _gc(x)
        _gc(fields)

        # Generate global LLM description via GPT-4.1 Vision (image directe)
        description = llm_describe(img, fname)

        m = {"document": fname, "analyzer": aid, "time_seconds": round(dt,1),
             "num_fields": len(fields), "num_fields_with_values": len(field_values),
             "markdown_len": len(md),
             "avg_confidence": round(sum(confs)/len(confs),4) if confs else None,
             "num_tables": len(block.get("tables",[])),
             "num_pages": len(block.get("pages",[])),
             "field_values": field_values,
             "description": description}
        all_metrics[aid].append(m)

        # Save JSON (with field values + description)
        enriched_result = res.copy()
        enriched_result["_extracted"] = {
            "field_values": field_values,
            "description": description
        }
        folder = os.path.join(OUTPUT_FOLDER, aid)
        os.makedirs(folder, exist_ok=True)
        with open(os.path.join(folder, f"{dname}.json"), "w", encoding="utf-8") as f:
            json.dump(enriched_result, f, indent=2, ensure_ascii=False)

        # Print summary
        status_icon = "📋" if field_values else "⚠️"
        print(f"  ✅ [{done}/{len(jobs)}] {fname}|{aid} {dt:.0f}s")
        print(f"     {status_icon} Fields with values: {len(field_values)}/{len(fields)}")
        if field_values:
            top = {k: (str(v)[:50] if not isinstance(v, (list,dict)) else f"[{type(v).__name__}]")
                   for k, v in list(field_values.items())[:5]}
            print(f"     📋 {top}")
        print(f"     📝 {description[:140]}")
    except Exception as e:
        dt = time.time() - t0
        print(f"  ❌ [{done}/{len(jobs)}] {fname}|{aid} {dt:.0f}s {str(e)[:80]}")
        all_metrics[aid].append({"document": fname, "analyzer": aid,
                                 "error": str(e), "time_seconds": round(dt,1)})

print(f"\n🎉 Done! Results in {OUTPUT_FOLDER}")

📤 Submitting all requests...
  ✓ [1/20] batch1-0001.jpg → prebuilt-invoice
  ✓ [2/20] batch1-0002.jpg → prebuilt-invoice
  ✓ [3/20] batch1-0003.jpg → prebuilt-invoice
  ✓ [4/20] batch1-0004.jpg → prebuilt-invoice
  ✓ [5/20] batch1-0005.jpg → prebuilt-invoice
  ✓ [6/20] batch1-0006.jpg → prebuilt-invoice
  ✓ [7/20] batch1-0007.jpg → prebuilt-invoice
  ✓ [8/20] batch1-0008.jpg → prebuilt-invoice
  ✓ [9/20] batch1-0009.jpg → prebuilt-invoice
  ✓ [10/20] batch1-0010.jpg → prebuilt-invoice
  ✓ [11/20] batch1-0011.jpg → prebuilt-invoice
  ✓ [12/20] batch1-0012.jpg → prebuilt-invoice
  ✓ [13/20] batch1-0013.jpg → prebuilt-invoice
  ✓ [14/20] batch1-0014.jpg → prebuilt-invoice
  ✓ [15/20] batch1-0015.jpg → prebuilt-invoice
  ✓ [16/20] batch1-0016.jpg → prebuilt-invoice
  ✓ [17/20] batch1-0017.jpg → prebuilt-invoice
  ✓ [18/20] batch1-0018.jpg → prebuilt-invoice
  ✓ [19/20] batch1-0019.jpg → prebuilt-invoice
  ✓ [20/20] batch1-0020.jpg → prebuilt-invoice

📤 20 jobs submitted. Waiting 8s before 